In [1]:
rm -rf obitools4
git clone https://github.com/metabarcoding/obitools4 

Cloning into 'obitools4'...
remote: Enumerating objects: 7574, done.
remote: Counting objects: 100% (591/591), done.
remote: Compressing objects: 100% (397/397), done.
remote: Total 7574 (delta 287), reused 384 (delta 148), pack-reused 6983 (from 2)
Receiving objects: 100% (7574/7574), 188.68 MiB | 29.37 MiB/s, done.
Resolving deltas: 100% (4721/4721), done.
Updating files: 100% (529/529), done.


In [2]:
### TEST FIXER LA VERSION 
cd obitools4
git reset --hard 996ec69
cd ..

HEAD is now at 996ec69 update the release notes for version 4.4.0


In [3]:
conda install -c conda-forge compilers=1.10 make=4.4.1 go=1.24.5 zlib=1.3.1 jq=1.8.1 --yes

Retrieving notices: done
Channels:
 - conda-forge
 - bioconda
Platform: linux-64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 25.3.1
    latest version: 25.5.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - compilers=1.10
    - go=1.24.5
    - jq=1.8.1
    - make=4.4.1
    - zlib=1.3.1


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _go_select-2.2.0           |            nocgo           5 KB  conda-forge
    binutils-2.43              |       h4852527_5          34 KB  conda-forge
    binutils_linux-64-2.43     |       h4852527_4          35 KB  conda-forge
    c-compiler-1.10.0          |       h2b85faf_0           7 KB  conda-forge
    ca-certificates-2025.7.14  |       hbd8a1cb_0         152 KB  conda-forge

In [4]:
# CHANGER LE LIEN DU GITLAB EN GITHUB
find . -type f -exec sed -i \
  -e 's|git\.metabarcoding\.org/obitools/obitools4/obitools4|github.com/metabarcoding/obitools4|g' \
  -e 's|git\.metabarcoding\.org/org-asm/org-asm\.git|github.com/metabarcoding/org-asm.git|g' \
  -e 's|git\.metabarcoding\.org/lecasofts/go/obitools|github.com/metabarcoding/obitools4|g' \
  -e 's|https://git\.metabarcoding\.org|https://github.com/metabarcoding|g' \
  {} +


In [5]:
export CC=$CONDA_PREFIX/bin/x86_64-conda-linux-gnu-cc
export CGO_ENABLED=1
export CGO_CFLAGS="-I$CONDA_PREFIX/include"
export CGO_LDFLAGS="-L$CONDA_PREFIX/lib"

In [ ]:
# Installation d'obitools V4
cd obitools4
make
cd ..

In [ ]:
## FORMATAGE DES DONNES / ESPACES RETIRE
shopt -s nullglob
for f in galaxy_inputs/inputfile/*.{fastq,fasta}; do
  [ -e "$f" ] || continue
  dir=$(dirname -- "$f")
  base=$(basename -- "$f")
  # on enlève les espaces
  newbase=${base// /_}
  newpath="$dir/$newbase"
  [ "$newpath" = "$f" ] && continue
  echo "Renomme: $f -> $newpath"
  mv -- "$f" "$newpath"
done

In [ ]:
jq '.inputfile[0].path |= gsub(" "; "_")' galaxy_inputs/galaxy_inputs.json >  galaxy_inputs/galaxy_inputs_fixed.json
json_path="galaxy_inputs/galaxy_inputs.json"

# input_file="galaxy_inputs/wolfdiet_obigrep.fastq" test injection d'input
input_file="$(grep -o '"path": *"[^"]*"' "$json_path" | head -n 1 | sed 's/.*"path": *"\([^"]*\)".*/\1/')"

echo "Input file path: $input_file"


filename=$(basename -- "$input_file")       # ex: myfile.fastq.gz
extension="${filename##*.}"                # ex: gz
filename="${filename%.*}"                  # ex: myfile.fastq ou myfile

echo "Nom de fichier (sans chemin, sans extension) : $filename"
echo "Extension détectée : $extension"

output_file="outputs/outputfile.$extension"
format=""

## Chemin ref file

ref_file=$(jq -r '.ref[0].path' galaxy_inputs/galaxy_inputs.json)

#Chemin paired end 

pairedwith_file=$(jq -r '.pairedwith[0].path' galaxy_inputs/galaxy_inputs.json)
pairedwith_file=$(printf "%s" "$pairedwith_file" | sed 's/[[:space:]]//g')

if [[ -n "$pairedwith_file"]]; then
    pairedwith="--paired-with $pairedwith_file"
    echo "variable pairedwith présente: $pairedwith"
else 
    pairedwith=""
fi

# geometric BOOLEEN -G
geom=$(jq -r '.geom' galaxy_inputs/galaxy_inputs.json)
if [[ "$geom" == "true" ]]; then
    geom="-G"
    echo "variable geomtric présente: $geom"
else
    geom=""
fi 

# SAVE DB 
save=$(jq -r '.save' galaxy_inputs/galaxy_inputs.json)
if [[ save == "true"]]; then
    save="--save-db outputs/ref_db"
else
    save=""
fi

Input file path: galaxy_inputs/inputfile/Suiteobigrep.fastqsanger.fastq
Nom de fichier (sans chemin, sans extension) : Suiteobigrep.fastqsanger
Extension détectée : fastq


In [ ]:
./obitools4/build/obitag -R $ref_file $save $input_file $pairedwith --out $output_file 

INFO[0000] Number of workers set 36                     
INFO[0000] Found 1 files to process                     
INFO[0000] /data/jwd06/pulsar_staging/86582615/inputs/dataset_d887f3b8-b64c-4bb3-b918-bf3e94557b11.dat mime type: text/fastq 
INFO[0000] Reading NGSFilter file: galaxy_inputs/ngs/db_v05_r117.fasta 
INFO[0000] NGSFilter configuration mimetype: text/fasta 
FATA[0000] NGSfilter reading file error: line 1 : invalid format : 
>AY189646 count=1; merged_taxid={9606: 1}; species_name=Homo sapiens; family=9604; family_name=Hominidae; scientific_name=Homo sapiens; taxid=9606; genus_name=Homo; forward_error=0; forward_tm=49.5; rank=species; seq_length_ori=568; species=9606; forward_primer=TTAGATACCCCACTATGC; reverse_primer=CAGAACAGGCTCCTCTAG; genus=9605; reverse_error=1; reverse_tm=48.3; strand=D;   Homo sapiens clone arCan119 12S ribosomal RNA gene, partial sequence; mitochondrial gene for mitochondrial product. 


: 1